# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [77]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
from pprint import pprint

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [51]:
path_csv = os.path.join("..","WeatherPy","city_weather_data.csv")
city_df = pd.read_csv(path_csv)     

# Drop null values
city_df.dropna(inplace=True)
city_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,jamestown,42.0970,-79.2353,66.02,73,4,3.00,US,1623643294
1,nanortalik,60.1432,-45.2371,34.32,92,48,5.41,GL,1623644002
2,atuona,-9.8000,-139.0333,79.11,81,85,22.88,PF,1623644003
3,kapaa,22.0752,-159.3190,82.38,78,74,1.01,US,1623644004
4,qaanaaq,77.4840,-69.3632,30.40,75,49,3.42,GL,1623644005


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [31]:
# Configure gmaps
gmaps.configure(api_key=g_key)

In [57]:
# Drop cities which does not fit weather conditions
hotel_df = city_df.drop(city_df.loc[(city_df["Max Temp"] >= 80) |
                                         (city_df["Max Temp"] <= 70) |
                                         (city_df["Wind Speed"] >= 10) |
                                         (city_df["Cloudiness"] != 0)].index) 

In [124]:
# Store 'Lat' and 'Lng' into  locations 
locations = city_df[["Lat", "Lng"]].astype('float')

# Store 'Humidity' in humidity
humidity = city_df["Humidity"].astype('float')
print(locations)
print(humidity)

         Lat       Lng
0    42.0970  -79.2353
1    60.1432  -45.2371
2    -9.8000 -139.0333
3    22.0752 -159.3190
4    77.4840  -69.3632
..       ...       ...
571  18.9048   -3.5265
572  14.3521    3.3168
573  26.7667 -109.6667
574   9.6000   -5.2000
575  59.7580   18.7050

[574 rows x 2 columns]
0      73.0
1      92.0
2      81.0
3      78.0
4      75.0
       ... 
571    21.0
572    38.0
573    71.0
574    79.0
575    59.0
Name: Humidity, Length: 574, dtype: float64


In [126]:
# Create a Humidity Heatmap layer
figure_layout = {
    'width': '400px',
    'height': '400px',
    'border': '1px solid black',
    'padding': '1px'
}
#layout=figure_layout
fig = gmaps.figure(zoom_level = 2, center=(30,45))
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                              dissipating=False, max_intensity=humidity.max(),
                                 point_radius = 1)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [52]:
# Drop cities which does not fit weather conditions
hotel_df = city_df.drop(city_df.loc[(city_df["Max Temp"] >= 80) |
                                         (city_df["Max Temp"] <= 70) |
                                         (city_df["Wind Speed"] >= 10) |
                                         (city_df["Cloudiness"] != 0)].index) 
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
76,vostok,46.4856,135.8833,73.04,31,0,7.65,RU,1623643388
117,sabha,27.0377,14.4283,74.98,28,0,8.12,LY,1623644149
139,bud,39.4470,-86.1758,70.34,60,0,5.68,US,1623644178
172,hamilton,39.1834,-84.5333,71.44,79,0,2.86,US,1623644009
377,san mauro torinese,45.1058,7.7545,73.38,68,0,3.00,IT,1623644493
398,taboshar,40.5686,69.6413,70.12,27,0,5.12,TJ,1623644518
462,alexandria,31.2156,29.9553,73.36,73,0,4.61,EG,1623644602
523,conde,-7.2597,-34.9075,71.06,94,0,5.75,BR,1623644678
538,gat,31.6100,34.7642,72.91,100,0,3.33,IL,1623644700
556,saint-pierre,-21.3393,55.4781,73.04,64,0,6.91,RE,1623644627


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [128]:
hotel_df["Hotel Name"] = ""
hotel_df["Hotel Address"] = ""
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name,Hotel Address
76,vostok,46.4856,135.8833,73.04,31,0,7.65,RU,1623643388,,
117,sabha,27.0377,14.4283,74.98,28,0,8.12,LY,1623644149,,
139,bud,39.4470,-86.1758,70.34,60,0,5.68,US,1623644178,,
172,hamilton,39.1834,-84.5333,71.44,79,0,2.86,US,1623644009,,
377,san mauro torinese,45.1058,7.7545,73.38,68,0,3.00,IT,1623644493,,
398,taboshar,40.5686,69.6413,70.12,27,0,5.12,TJ,1623644518,,
462,alexandria,31.2156,29.9553,73.36,73,0,4.61,EG,1623644602,,
523,conde,-7.2597,-34.9075,71.06,94,0,5.75,BR,1623644678,,
538,gat,31.6100,34.7642,72.91,100,0,3.33,IL,1623644700,,
556,saint-pierre,-21.3393,55.4781,73.04,64,0,6.91,RE,1623644627,,


In [132]:
# Store 'Lat' and 'Lng' into  locations 
locations = hotel_df[["Lat", "Lng"]].astype('float')

# Store 'Humidity' in humidity
humidity = hotel_df["Humidity"].astype('float')

# geocoordinates
target_coordinates = "46.4856, 135.8833"
target_search = "Chinese"
target_radius = 8000
target_type = "restaurant"

# set up a parameters dictionary
params = {
    "location": target_coordinates,
    "radius": 5000,
    "type": 'lodging',
    #"language": 'en',
    "key": g_key
}

# base url
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

for index, row in hotel_df.iterrows():
    # get lat, lng from df
    lat = row["Lat"]
    lng = row["Lng"]

    # change location in each iteration
    params["location"] = f"{lat},{lng}"

    # run a request using our params dictionary
    hotel_data = requests.get(base_url, params=params).json()
    #pprint(hotel_data)
    try:
        hotel_df.loc[index, "Hotel Name"] = hotel_data['results'][0]['name']
        hotel_df.loc[index, "Hotel Adress"] = hotel_data['results'][0]['vicinity']
    except:
        print("Hotel not found for " + hotel_df.loc[index, "City"]) 
hotel_df
# print(response.json())

Hotel not found for vostok
Hotel not found for taboshar


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name,Hotel Address,Hotel Adress
76,vostok,46.4856,135.8833,73.04,31,0,7.65,RU,1623643388,,,NaN
117,sabha,27.0377,14.4283,74.98,28,0,8.12,LY,1623644149,Foundation Taaj Asala for Historical Research ...,,Sabhā
139,bud,39.4470,-86.1758,70.34,60,0,5.68,US,1623644178,Ted McQuinn Trust,,"2062 West Road 100 South, Bargersville"
172,hamilton,39.1834,-84.5333,71.44,79,0,2.86,US,1623644009,Six Acres Bed & Breakfast,,"5350 Hamilton Avenue, Cincinnati"
377,san mauro torinese,45.1058,7.7545,73.38,68,0,3.00,IT,1623644493,Parco Sassi Hotel,,"Via Tommaso Agudio, 31, Torino"
398,taboshar,40.5686,69.6413,70.12,27,0,5.12,TJ,1623644518,,,NaN
462,alexandria,31.2156,29.9553,73.36,73,0,4.61,EG,1623644602,Royal Crown Hotel,,"Cliopatra Bridge, AR Riyadah"
523,conde,-7.2597,-34.9075,71.06,94,0,5.75,BR,1623644678,Pousada Beija Flor,,Rua Maria Amélia - s/n
538,gat,31.6100,34.7642,72.91,100,0,3.33,IL,1623644700,OlusHome,,"ציבעוני 68, ניר ח""ן"
556,saint-pierre,-21.3393,55.4781,73.04,64,0,6.91,RE,1623644627,Lindsey Hôtel,,21 Rue Francois Isautier


'Franklin House'
'Boise Guest House'
'The Riverside Hotel, Bw Premier Collection'
'The Riverside Hotel'
'The Pink Guest House'
'URBAN by Hotel Inn'
'Hyatt Place Boise/Downtown'
'The Anniversary Inn'
'Red Lion Hotel Boise Downtowner'
'Ramada by Wyndham Boise'


In [134]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]


# Create a marker_layer using the poverty list to fill the info box
#fig = gmaps.figure()
markers = gmaps.marker_layer(locations,
    info_box_content=hotel_info)
fig.add_layer(markers)
fig

Figure(layout=FigureLayout(height='420px'))

In [ ]:
# Add marker layer ontop of heat map


# Display figure
